# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents
4 agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",  # set manager
                           temperature=0.7),
    process=Process.hierarchical,  # 1 manager that delegates work to agents to perform tasks
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should delegate tasks to the Data Analyst and Trading Strategy Developer to ensure we are using statistical modeling and machine learning effectively.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL and analyze it using statistical modeling and machine learning

I should start by reading the website content to gather information on the machine learning models used to predict AAPL stock prices.

Action: Read website content
Action Input: {"website_url": "https://www.examplewebsite.com/machine-learning-models-predict-AAPL-stock-prices"} 

Page not found – Example Website
Skip to content 
Example Website
Menu 
Menu 
Home
Blog
About
Contact
Oops! That page can’t be found.
It looks like nothing was found at this location. Maybe try searching?
Search for:
Search
RECENT POSTS
By 
Footer Widget 1
This is just an example text widget. You can add any kind of widget in here.
Mauris lacinia rhoncus lacus, eget bibendum ex gravida ac. Proin sed mauris ipsum. Sed eu commodo ipsum.
Footer Widget 2
This is just an example text widget. You can add any kind of widget in here.
Mauris lacinia rhoncus lacus, eget bibendum ex gravida ac. Proin sed mauris ipsum. Sed eu commodo ipsum.
Facebook
Twitter
Linkedin
© 2024 Example Website • Built with GeneratePress

Final 

Final Answer: Here are some approved trading strategies for AAPL based on the search results:

1. Strategies & Applications Behind The 50-Day EMA: The 50-day EMA can be used for price prediction, position choice, and strategy building.
2. How to Trade Apple (AAPL) in the First Half of 2021: The trading strategy suggests buying Apple stock on weakness to the semiannual and annual value levels at $114.04 and $97.31 and reducing holdings on strength.
3. Watch Out for These Levels in Apple's Stock Price: This strategy focuses on identifying specific levels in Apple's stock price to make informed trading decisions.
4. Gap Trading: How to Play the Gap: This strategy involves utilizing the phenomenon of price gaps in Apple's stock to inform trading decisions.
5. Investing in Apple Stock (AAPL): This provides insights into investing in Apple stock and the growth strategy of purchasing small tech companies.
6. Options Trading: How to Trade Stock Options in 5 Steps: This article covers different

Thought: I now can give a great answer
Final Answer: Based on the search results, there are various trading strategies for AAPL including options strategies, day trading strategies, and stock trading strategies. Some of the strategies mentioned include Straddle, Buy-Write, Risk Reversal, Call Spread, long call debit spreads, and analyzing chart patterns. These strategies involve different levels of risk and potential returns. To evaluate the risks associated with these trading strategies for AAPL, a detailed analysis of each strategy's potential downside, market conditions, and historical performance would be necessary. Mitigation strategies could include setting stop-loss orders, diversifying the portfolio, and closely monitoring market trends and news related to AAPL.

> Finished chain.
 

Based on the search results, there are various trading strategies for AAPL including options strategies, day trading strategies, and stock trading strategies. Some of the strategies mentioned inclu

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

1. Strategies & Applications Behind The 50-Day EMA:
- Potential Risks: Relying solely on the 50-day EMA for price prediction could lead to false signals and missed opportunities.
- Mitigation Strategies: Combine the 50-day EMA with other technical indicators for confirmation, and set stop-loss orders to limit losses.

2. How to Trade Apple (AAPL) in the First Half of 2021:
- Potential Risks: Buying on weakness and selling on strength may not always align with market conditions, leading to missed opportunities or losses.
- Mitigation Strategies: Conduct thorough fundamental analysis alongside technical analysis, and consider setting trailing stops to protect profits.

3. Watch Out for These Levels in Apple's Stock Price:
- Potential Risks: Relying solely on specific levels may not account for sudden market movements or external factors impacting AAPL's stock price.
- Mitigation Strategies: Use a combination of support and resistance levels, trend analysis, and market news to make informed decisions.

4. Gap Trading: How to Play the Gap:
- Potential Risks: Gap trading can be volatile and risky, especially if gaps are not filled or if market conditions change suddenly.
- Mitigation Strategies: Implement strict risk management practices, such as setting tight stop-loss orders and avoiding overleveraging.

5. Investing in Apple Stock (AAPL):
- Potential Risks: Investing in individual stocks like AAPL carries inherent risks, including volatility, market sentiment shifts, and company-specific news.
- Mitigation Strategies: Diversify the investment portfolio, conduct thorough research on AAPL's financials, and consider long-term investment goals.

6. Options Trading: How to Trade Stock Options in 5 Steps:
- Potential Risks: Options trading involves leverage and can lead to substantial losses if not managed properly.
- Mitigation Strategies: Educate yourself on options trading strategies, use risk management tools like stop-loss orders, and start with small position sizes.

7. What Moves Apple's Stock Price?:
- Potential Risks: Focusing solely on long-term drivers may not account for short-term market fluctuations and sentiment shifts.
- Mitigation Strategies: Stay informed on both long-term trends and short-term market news, and adjust trading strategies accordingly.

8. How a Protective Collar Options Strategy Works:
- Potential Risks: The protective collar options strategy may limit potential gains if AAPL's stock price surpasses the options' strike prices.
- Mitigation Strategies: Consider adjusting strike prices based on market conditions and the stock's volatility, and monitor the options closely for potential adjustments.

9. Apple Traders Pile In To Bet on 12% Gain:
- Potential Risks: Following trends blindly without conducting proper analysis may lead to losses if the trend reverses unexpectedly.
- Mitigation Strategies: Use trend analysis alongside fundamental and technical analysis to confirm trading decisions, and be prepared to exit positions if trends change.

10. Apple's Stock Outlook Overwhelmingly Bullish, Options:
- Potential Risks: Overly bullish sentiments can lead to complacency and missed opportunities to adjust trading strategies.
- Mitigation Strategies: Maintain a balanced view of market sentiments, conduct thorough analysis, and be prepared to adjust strategies based on changing market conditions.

Overall, the risks associated with the proposed trading strategies for AAPL vary depending on the strategy's complexity, market conditions, and individual risk tolerance. To mitigate these risks, it is essential to conduct thorough analysis, use risk management tools, and stay informed on market trends and news related to AAPL.